# R notebook for running DecontX

20210121

Stephen Fleming

How to run DecontX on our data... the details of this took a long time to figure out: thank you to Ayshwarya Subramanian and Orr Ashenberg for help figuring this out.

In [1]:
import rpy2
%load_ext rpy2.ipython

## Installation

In [2]:
# %%R

# BiocManager::install("celda")

In [3]:
%%R

library(celda)

In [4]:
# %%R

# BiocManager::install("DropletUtils")

In [5]:
%%R

library(DropletUtils)

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: GenomicRanges

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find

## Get data

In [9]:
%%R

file = '/home/jupyter-user/data/cellranger/02-P005175-S053-R01.h5'
file

[1] "/home/jupyter-user/data/cellranger/02-P005175-S053-R01.h5"


In [12]:
!ls -lh /home/jupyter-user/data/cellranger/mtx/02-P005175-S053-R01

total 38M
-rw-rw-r-- 1 jupyter-user users  47K Jan 22 00:27 barcodes.tsv.gz
-rw-rw-r-- 1 jupyter-user users 341K Jan 22 00:27 features.tsv.gz
-rw-rw-r-- 1 jupyter-user users  37M Jan 22 00:27 matrix.mtx.gz


## Running the tool

In [6]:
%%R

sce = read10xCounts('/home/jupyter-user/data/cellranger/mtx/02-P005175-S053-R01', sample.names="D1_heart")
sce

class: SingleCellExperiment 
dim: 33567 9344 
metadata(1): Samples
assays(1): counts
rownames(33567): GRCh38premrna_ENSG00000243485
  GRCh38premrna_ENSG00000237613 ... SARSCoV2______SARSCoV2_3prime
  SARSCoV2______SARSCoV2_NegStrand
rowData names(3): ID Symbol Type
colnames: NULL
colData names(2): Sample Barcode
reducedDimNames(0):
spikeNames(0):
altExpNames(0):


In [7]:
%%R

colnames(sce) <- paste(sce$Sample, sce$Barcode, sep = "_")
rownames(sce) <- rowData(sce)$Symbol

In [8]:
%%R

keep_feature <- rowSums(counts(sce) > 0) > 0
sce <- sce[keep_feature, ]

In [9]:
%%R

str(sce@assays@data$counts)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:10297935] 40 117 156 167 179 187 212 222 223 250 ...
  ..@ p       : int [1:9345] 0 1380 3715 5073 5618 7681 8372 9533 10736 11293 ...
  ..@ Dim     : int [1:2] 26312 9344
  ..@ Dimnames:List of 2
  .. ..$ : NULL
  .. ..$ : NULL
  ..@ x       : num [1:10297935] 1 1 1 1 2 1 1 1 1 1 ...
  ..@ factors : list()


In [10]:
%%R

str(as.matrix(counts(sce)))

 num [1:26312, 1:9344] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:26312] "GRCh38premrna_AL627309.1" "GRCh38premrna_AL627309.2" "GRCh38premrna_AC114498.1" "GRCh38premrna_AL669831.5" ...
  ..$ : chr [1:9344] "D1_heart_AAACCCAAGATACCAA-1" "D1_heart_AAACCCACACGCTGTG-1" "D1_heart_AAACCCATCAAGGCTT-1" "D1_heart_AAACCCATCGCGTGCA-1" ...


In [11]:
%%R

X <- as.matrix(counts(sce))

In [12]:
%%R

X <- apply(X, 1:2, as.integer)

In [13]:
%%R

str(X)

 int [1:26312, 1:9344] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:26312] "GRCh38premrna_AL627309.1" "GRCh38premrna_AL627309.2" "GRCh38premrna_AC114498.1" "GRCh38premrna_AL669831.5" ...
  ..$ : chr [1:9344] "D1_heart_AAACCCAAGATACCAA-1" "D1_heart_AAACCCACACGCTGTG-1" "D1_heart_AAACCCATCAAGGCTT-1" "D1_heart_AAACCCATCGCGTGCA-1" ...


In [ ]:
%%time
%%R

result <- celda::decontX(X)

R[write to console]: --------------------------------------------------

R[write to console]: Start DecontX. Decontamination

R[write to console]: --------------------------------------------------



## Save output

In [ ]:
%%R

write.csv(result$resList$logLikelihood, 
          '/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/logLikelihood.csv', quote=FALSE)
write.csv(result$resList$theta, 
          '/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/theta.csv', quote=FALSE)
write.csv(result$resList$estConp, 
          '/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/estConp.csv', quote=FALSE)
write.csv(result$resList$estNativeCounts, 
          '/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/estNativeCounts.csv', quote=FALSE)

In [ ]:
%%R

# try to save in sparse format

out <- Matrix(result$resList$estNativeCounts, sparse = TRUE)

writeMM(out, file = '/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/mtx/matrix.mtx')

write.csv2(colnames(out), file ="/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/mtx/barcodes.tsv", 
           row.names=FALSE, col.names=FALSE, sep="\t", quote=FALSE)
write.csv2(rownames(out), file ="/home/jupyter-user/analysis/decontx/02-P005175-S053-R01/mtx/genes.tsv", 
           row.names=FALSE, col.names=FALSE, sep="\t", quote=FALSE)